In [2]:
import time
import re
import random
# from datetime import datetime, date, time, timedelta
import pickle
import requests
from requests.auth import HTTPBasicAuth
import undetected_chromedriver
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys 
import warnings
warnings.filterwarnings("ignore")

В этом ноутбуке разбираем 1 часть парсера - получение id одного отделения банка для одного города, 2 часть - формирования списка id всех отделений банка для этого города

#### 1. получить id любого отделения ВТБ на яндекс картах в городе Елец 

зарегистрироваться получить бесплатный apikey в api поиска по организациям https://developer.tech.yandex.ru/services/12 и записать его в переменную apikey строкой


PS: можешь выбрать любой банк/город, чем меньше город, тем меньше там отделений - быстрее работает



In [5]:
bank_name = 'alfa_bank'
city_name = 'Алапаевск' 
city_name = city_name.lower()
city_name

'алапаевск'

In [6]:
params = dict(
text=f'сбербанк {city_name}',
type='biz',
lang='en_US',
apikey='your_key'
)
url = 'https://search-maps.yandex.ru/v1/'
resp = requests.get(url=url, params=params)
data = resp.json()

#### 2. разобрать функцию get_yndx_id_from_chain, запустить её для найденного банка и id отделения

обрати внимание, что имя банка ВТБ передаём как bank_vtb - разобраться почему. и оцени результат работы функции - все ли банки найдены?

In [15]:
def find_between(s, first, last):
    """находит строку между символами first и last"""
    try:
        regex = rf'{first}(.*?){last}'
        return re.findall(regex, s)
    except ValueError:
        return -1


def get_id_from_page(page, bank_name):

    """вытаскивает id банка яндекс карта из page"""
    first = f'search-snippet-view__link-overlay _focusable" href="/maps/org/{bank_name}/'

    last = '/" tabindex="'
    yndx_idx = find_between(page, first, last)
    print(f'loaded {len(yndx_idx)} ids')

    return yndx_idx


def get_yndx_id_from_chain(yndx_bank_id, bank_name):

    """формирует список ссыллок-банков для текущего банка id из яндекс карт из раздела Филиалы"""
    
    opts = undetected_chromedriver.ChromeOptions()
    opts.add_argument("--disable-renderer-backgrounding")
    opts.add_argument("--disable-extensions")
    opts.add_argument('--no-sandbox')
    opts.add_argument('--disable-dev-shm-usage')
    opts.add_argument('headless')
    opts.add_argument('--disable-gpu')
    driver = undetected_chromedriver.Chrome(options=opts)
    
    url = f'https://yandex.ru/maps/org/{bank_name}/{yndx_bank_id}/chain/'
    driver.get(url)

    N = round(random.uniform(13.1, 19.9), 2) # was higher
    print(f'sleep for {N}')
    time.sleep(N)
    
    t = 'business-tab-wrapper' 
    elements = driver.find_elements(By.CLASS_NAME, t)
    print(f'LEN {len(elements)}')
    
    if len(elements) > 1:


        # без этого не работает
        elements = driver.find_elements(By.CLASS_NAME, t)
    
        seen = []
    
        while True:
            # скролл
            driver.execute_script("arguments[0].scrollIntoView();", elements[-1]);

            N = round(random.uniform(13.1, 19.9), 2) # was higher
            print(f'sleep for {N}')
            time.sleep(N)
            elements = driver.find_elements(By.CLASS_NAME, t)
            
            if len(elements) < 1:
                T = round(random.uniform(10.1, 19.9), 2)
                print(f'sleep again for {T}')
                time.sleep(T)
    
            page3 = elements[3].get_attribute('innerHTML')
            yndx_idx = get_id_from_page(page3, bank_name)
            
            last_size = len(yndx_idx)
            seen.append(last_size)
            
            if len(set(seen)) < len(seen):
                break
    
    else:
        print('driver stopped')
        driver.close()
        driver.quit()
        
    driver.close()
    driver.quit() 
    
    return yndx_idx


#### 3. придумать, как вместо отделений банков вытаскивать банкоматы - реализовать (исправить предыдущие две функции)